# Conformational and Markov state model analysis of heparin-analogue pentasaccharides

Correspondence: S. Kashif Sadiq (kashif.sadiq@embl.de) Affiliation: 1. Heidelberg Institute for Theoretical Studies, HITS gGmbH 2. European Moelcular Biology Laboratory
        
This notebook contains molecular dynamnics (MD) simulation and Markov state model (MSM) analysis for the manuscript:

Balogh, Gabor; Gyöngyösi, Tamás; Timári, István; Herczeg, Mihály; Borbás, Anikó; Sadiq, S. Kashif; Fehér, Krisztina; Kövér, Katalin, 
Conformational Analysis of Heparin-Analogue Pentasaccharides by Nuclear Magnetic Resonance Spectroscopy and Molecular Dynamics Simulations (2021), 
Journal of Chemical Information and Modeling, Accepted.

Here we analyze the conformational dynamics of 3 analogues of heparin pentasaccharrides: 1 - idraparinux, 2- disulfonic acid analogue and 3 - trisulfonic acid analogue.

Each analogue has 5 rings - D,E,F,G,H. The rings can theoretically exhibit internal puckering measured by the Cremer-Pople angle (theta,phi), but also global conformational transitions of the analogue can occur via dihedral angle changes of the interglycosidic linkages.

There are two bonds between each ring that form the interglycosidic linkage via a cental oxygen atom. In the nomenclature here, for example, going from D-E rings, the linkages are D-O-E and the dihedral angle around D-O is termed psi, the dihedral angle around O-E is termed phi. 

psi_DE = D(H1):D(C1):E(O):E(C4)
phi_DE = D(C1):E(O):E(C4):E(H4)


### Import Modules and Functions

In [ ]:
%pylab inline
from pentamd import *
from msmanalysis import *


### Make Directories

In [ ]:
# Make figures directory
fig_dir='figures'
if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)

## Load Trajectories 

Default path data is ../, with folders 1/ 2/ and 3/ corresponding to the 3 analogs and subdirectory filtered/ assumed to contain 10 replicas of production sims:
1_md.nc, 2_md.nc etc. 

U1,U2,U3 are each a list of MDA universes where each contains a replica trajectory of the corresponding analog 
with 200,000 timepoints (2 microseconds each, interval of 10 ps)


In [ ]:
path='../'
U1 = U_ensemble(path,1)
U2 = U_ensemble(path,2)
U3 = U_ensemble(path,3)

## Compute Average 1H-1H nuclei distances - for comparison with NOE

#### List of 1H-1H pairs

In [ ]:
nuclei_list=[['D1', 'E4'],
            ['D1', 'E5'],
             ['D1', 'D3'],
             ['D1', 'E3'],
             ['D61', 'D5'],
             ['E1', 'F61'],
             ['E1', 'F4'],
             ['E1', 'E4'],
             ['E1', 'E3'],
             ['F1', 'F3'],
             ['F1', 'F2'],
             ['F1', 'G4'],
             ['F1', 'G3'],
             ['G1', 'H3'],
             ['G1', 'H61'],
             ['G1', 'H62'],
             ['G1', 'H5'],
             ['G1', 'H4'],
             ['G1', 'G3'],
             ['G1', 'G2'],
             ['G2', 'G3'],
             ['G4', 'G3'],
             ['G5', 'G4'],
             ['G5', 'H4'],
            ['G5', 'G2']]

#### Compute Table of Average Distances for All Analogs

In [ ]:
resid_dict = {'D': 5, 'E': 4, 'F': 3, 'G': 2, 'H': 1}
Uall=[U1,U2,U3]
D=np.empty((25,0))
for U in Uall:
    dist=nuclei_list_avg_dist(U,resid_dict, nuclei_list)
    D=np.hstack(((D,dist)))

Dtable=np.hstack((np.array(nuclei_list),np.round(D,2) ))    

#### Save data files

In [ ]:
# Make distance directory
dist_dir='hhdist'
filename='/avg_1h_dist_skip10'
if not os.path.exists(dist_dir):
    os.mkdir(dist_dir)
np.set_printoptions(precision=2)
#Save files
np.savetxt(str(dist_dir)+'/'+str(filename)+'.dat',D, fmt='%6.2f',delimiter= ' ')
np.savetxt(str(dist_dir)+'/'+str(filename)+'.txt',Dtable, fmt='%s',delimiter= ' ')
#Print out Table of distances - nuclei 1, nuclei 2, mean std analogs 1 2 3 respectively 
#print(D)
#print(Dtable)

## RMSD Analysis

#### Select Reference Structure and Alignement Criteria

In [ ]:
ref_prmtop='pdbs/reference.pdb'
ref_pdb='pdbs/reference.pdb'
selection="not protein and name C1 C2 C3 C4 C5 O4 O5"


#### RMSD of each replica of each analog

In [ ]:
R1=ensemble_rmsd(path,1,ref_prmtop,ref_pdb,selection)
R2=ensemble_rmsd(path,2,ref_prmtop,ref_pdb,selection)
R3=ensemble_rmsd(path,3,ref_prmtop,ref_pdb,selection)
allR=[R1,R2,R3]

#### PLot RMSD Timeseries

In [ ]:
#Plot RMSD timeseries figure
fig,plt=plot_rmsd_timeseries(plt,allR)

# Make rmsd directory
rmsd_dir=str(fig_dir)+'/rmsd'
if not os.path.exists(rmsd_dir):
    os.mkdir(rmsd_dir)
#save figure
fig.savefig(str(rmsd_dir)+'/rmsd_dots.eps')
fig.savefig(str(rmsd_dir)+'/rmsd_dots.png')

### Plot RMSD distribution

In [ ]:
#Plot RMSD distribution figure
fig,plt=plot_rmsd_distribution(plt,allR)

# Make rmsd directory
rmsd_dir=str(fig_dir)+'/rmsd'
if not os.path.exists(rmsd_dir):
    os.mkdir(rmsd_dir)
#save figure
fig.savefig(str(rmsd_dir)+'/rmsd_dist.eps')
fig.savefig(str(rmsd_dir)+'/rmsd_dist.png')

## Ring puckering and Glycosidic linkage

In [ ]:
# Make data directory
data_dir='data'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)


#### Example single trajectory

In [ ]:
glink=["H1", "C1", "O4", "C4", "H4"]
ring_ordering=[0,0,-1,-1,-1]
#Analog1
gl1=glycosidic_link_dihedral_angles(U1[0],glink,ring_ordering)

#Plot single trajectory figure 
fig,plt=plot_single_trajectory(plt,gl1[:,1])

### Compute Glycosidic linkage Phi/Psi angles

Takes some time to compute all 10 replicas!! Existing data can be loaded instead from below

In [ ]:
glink=["H1", "C1", "O4", "C4", "H4"]
ring_ordering=[0,0,-1,-1,-1] 


for i in range(10):
    u=U1[i]
    gl=glycosidic_link_dihedral_angles(u,glink,ring_ordering)
    np.savetxt(str(data_dir)+'/GL_1_'+str(i+1)+'.dat', gl, fmt='%9.5f', delimiter=' ')    
    print('Analog 1: Replica '+str(i+1)+' Complete')
    
    u=U2[i]
    gl=glycosidic_link_dihedral_angles(u,glink,ring_ordering)
    np.savetxt(str(data_dir)+'/GL_2_'+str(i+1)+'.dat', gl, fmt='%9.5f', delimiter=' ')    
    print('Analog 2: Replica '+str(i+1)+' Complete')
    
    u=U3[i]
    gl=glycosidic_link_dihedral_angles(u,glink,ring_ordering)
    np.savetxt(str(data_dir)+'/GL_3_'+str(i+1)+'.dat', gl, fmt='%9.5f', delimiter=' ')        
    print('Analog 3: Replica '+str(i+1)+' Complete')   
    
    

### Compute Cremer-Pople Parameters

Takes some time to compute all 10 replicas!! Existing data can be loaded instead from below

In [ ]:
resids=[x for x in range(5,0,-1)]
for i in range(2,10):
    u=U1[i]
    cp=cremer_pople_analysis(u,resids)    
    np.savetxt(str(data_dir)+'/CP_1_'+str(i+1)+'.dat', cp, fmt='%9.5f', delimiter=' ')
    print('Analog 1: Replica '+str(i+1)+' Complete')
    
    u=U2[i]
    cp=cremer_pople_analysis(u,resids)    
    np.savetxt(str(data_dir)+'/CP_2_'+str(i+1)+'.dat', cp, fmt='%9.5f', delimiter=' ')
    print('Analog 2: Replica '+str(i+1)+' Complete')
    
    u=U3[i]
    cp=cremer_pople_analysis(u,resids)    
    np.savetxt(str(data_dir)+'/CP_3_'+str(i+1)+'.dat', cp, fmt='%9.5f', delimiter=' ')    
    print('Analog 3: Replica '+str(i+1)+' Complete')
     

### Load Pre-computed Cremer-Pople Parameters and Glycosdic linkage Dihedrals for all Replicas

Load preexisting Cremer Pople and Glycosidic data here - Needs to be placed in the ./data directory and with filenmae format CP_1.dat, CP_2.dat, GL_1.dat, GL_2.dat etc  
If you don't need to compute RMSD Analysis or Cremer-Pople Analysis then you can start from here after loading functions and modules

In [ ]:
#Cremer-Pople
CP1=load_data_array(analog=1,rep_start=1,rep_end=10,data_type='CP')
CP2=load_data_array(analog=2,rep_start=1,rep_end=10,data_type='CP')
CP3=load_data_array(analog=3,rep_start=1,rep_end=10,data_type='CP')
#Glycosidic linkage
GL1=load_data_array(analog=1,rep_start=1,rep_end=10,data_type='GL')
GL2=load_data_array(analog=2,rep_start=1,rep_end=10,data_type='GL')
GL3=load_data_array(analog=3,rep_start=1,rep_end=10,data_type='GL')

#### Join All feature data in Y format for pyemma
Y1=join_in_Y_format(CP1,GL1)
Y2=join_in_Y_format(CP2,GL2)
Y3=join_in_Y_format(CP3,GL3)

#### Concatenate all replicas per analog
Z1=concat_replicas(Y1)
Z2=concat_replicas(Y2)
Z3=concat_replicas(Y3)


Zlabels=[r"$\theta_{cp}$ ($^{\circ}$)", "$\phi_{cp}$ ($^{\circ}$)",
         r"$\theta_{cp}$ ($^{\circ}$)", "$\phi_{cp}$ ($^{\circ}$)",
         r"$\theta_{cp}$ ($^{\circ}$)", "$\phi_{cp}$ ($^{\circ}$)",
         r"$\theta_{cp}$ ($^{\circ}$)", "$\phi_{cp}$ ($^{\circ}$)",
         r"$\theta_{cp}$ ($^{\circ}$)", "$\phi_{cp}$ ($^{\circ}$)",
         "$\phi_{gl}$ ($^{\circ}$)", "$\psi_{gl}$ ($^{\circ}$)",
         "$\phi_{gl}$ ($^{\circ}$)", "$\psi_{gl}$ ($^{\circ}$)",
         "$\phi_{gl}$ ($^{\circ}$)", "$\psi_{gl}$ ($^{\circ}$)",
         "$\phi_{gl}$ ($^{\circ}$)", "$\psi_{gl}$ ($^{\circ}$)"]


#Combine analogues in a list
all_Z=[Z1,Z2,Z3]
all_CP=[CP1,CP2,CP3]
all_GL=[GL1,GL2,GL3]



### Plot Glycosidic and Cremer-Pople timeseries data for a given analog

In [ ]:
#Plot Analog
analog=3
fig,plt=plot_cp_gl_timeseries(plt,all_CP[analog-1],all_GL[analog-1])

# Make rmsd directory
cpgl_dir=str(fig_dir)+'/cpgl'
if not os.path.exists(cpgl_dir):
    os.mkdir(cpgl_dir)
#save figure
fig.savefig(str(cpgl_dir)+'/analog'+str(analog)+'_cpgl_dots.eps')
fig.savefig(str(cpgl_dir)+'/analog'+str(analog)+'_cpg1_dots.png')


### Plot 2D Densities of Cremer-Pople / Glycosidic linkages

#### Single 2D Density Plot

In [ ]:
analog=1
Z=all_Z[analog-1]
ring_id=1
dim=[2*ring_id+1,2*ring_id]
plt=plot_weighted_free_energy_landscape_range(Z,plt,dim[0],dim[1],Zlabels,contour_label=False,
                                              cmap='Spectral',cbar=True,x_ticksep=20,y_ticksep=10,cbar_label="-k$_{B}$Tln(p) (kcal/mol)",
                                             fsize_cbar=(20,15))

plt.grid()


#### Combined Cremer-Pople and Glycosidic linkage density Arrays

In [ ]:

#Plot CP GL 2D density 
fig,plt=plot_cp_gl_density2D(plt,Zlabels,all_Z)

# Make cpgl directory
cpgl_dir=str(fig_dir)+'/cpgl'
if not os.path.exists(cpgl_dir):
    os.mkdir(cpgl_dir)
#save figure
fig.savefig(str(cpgl_dir)+'/cpgl_density2D.eps')
fig.savefig(str(cpgl_dir)+'/cpg1_density2D.png')


#### Cremer Pople 2D density array plot only

In [ ]:

#Plot CP 2D density 
fig,plt=plot_cp_density2D(plt,Zlabels,all_Z)

# Make cpgl directory
cpgl_dir=str(fig_dir)+'/cpgl'
if not os.path.exists(cpgl_dir):
    os.mkdir(cpgl_dir)
#save figure
fig.savefig(str(cpgl_dir)+'/cp_density2D.eps')
fig.savefig(str(cpgl_dir)+'/cp_density2D.png')


#### Glycosidic linkage 2D density array plot only 

In [ ]:

#Plot CP 2D density 
fig,plt=plot_gl_density2D(plt,Zlabels,all_Z)

# Make cpgl directory
cpgl_dir=str(fig_dir)+'/cpgl'
if not os.path.exists(cpgl_dir):
    os.mkdir(cpgl_dir)
#save figure
fig.savefig(str(cpgl_dir)+'/gl_density2D.eps')
fig.savefig(str(cpgl_dir)+'/gl_density2D.png')



### Average Glycosidic Torsion angles 

In [ ]:
np.set_printoptions(precision=2)
glyco_mean=np.transpose(np.vstack(( np.mean(Z1,axis=0)[10:], np.mean(Z2,axis=0)[10:], np.mean(Z3,axis=0)[10:])) )
glyco_std=np.transpose(np.vstack(( np.std(Z1,axis=0)[10:], np.std(Z2,axis=0)[10:], np.std(Z3,axis=0)[10:]   )) )

#print(glyco_mean)
#print(glyco_std)

# Make gl_torsion directory
gltor_dir='gl_torsion'
if not os.path.exists(gltor_dir):
    os.mkdir(gltor_dir)

np.savetxt(str(gltor_dir)+'/glyco_mean.dat',glyco_mean, fmt='%6.2f',delimiter=' ')
np.savetxt(str(gltor_dir)+'/glyco_std.dat',glyco_std, fmt='%6.2f',delimiter=' ')


### Ring Linker Pearson Correlation

In [ ]:
#Reorder the columns of Z that contains CP and GL info in the linear sequence along the pentasaccharide
data_reorder=[]
for gl_id in range(4):
    data_reorder+=[2*gl_id, 2*gl_id+10, 2*gl_id+11]
data_reorder+=[8]
#print(data_reorder)

#Choose analog and compute Pearson corelation matrix from corresponding Z (CP,GL) data 
analog=1
Z_mat=cpgl_pearsoncorr_matrix(all_Z[analog-1],data_reorder)

# Plot Pearson correlation matrix
fig,plt=plot_cpgl_pearsoncorr_matrix(plt,Z_mat)

# Make pearson directory
pearson_dir=str(fig_dir)+'/pearson'
if not os.path.exists(pearson_dir):
    os.mkdir(pearson_dir)
#save figure
fig.savefig(str(pearson_dir)+'/pearsoncorrelation_matrix.eps')
fig.savefig(str(pearson_dir)+'/pearsoncorrelation_matrix.png')


### Binary Conformational Clustering using E and G rings

In [ ]:
np.set_printoptions(precision=6)

#choose_analog
analog=3

#Compute condition matrix
cond_mat,cond_len,Zcluster=state_selection(all_Z[analog-1])

#Select specific data point indices corresponding to stringent range corresponding to each binary ring state
E0=ring_binary_state_selection(Zcluster, 0, [[18,22], [0,5]])
E1=ring_binary_state_selection(Zcluster, 0, [[80,90], [70,90]])
G0=ring_binary_state_selection(Zcluster, 1, [[90,100], [135,140]])
G1=ring_binary_state_selection(Zcluster, 1, [[165,170], [115,120]])


# Select indices of coexisting state combinations
S=[np.intersect1d(E0,G0),np.intersect1d(E0,G1),np.intersect1d(E1,G0),np.intersect1d(E1,G1)]
representative_frames=np.empty((0,2))
for i in range(len(S)):
    #Check to see if selected combined state has any indices - if not, use the more relaxed definition of the state from binary cluster selection
    if S[i].size==0:
        S[i]=Zcluster[cond_mat[i],0].astype(int)
    #Convert indices to replica and frame ids - so they can be saved and later selected in VMD
    S[i]=convert_to_replica_frame(S[i])

    representative_frames=np.vstack((representative_frames,S[i][0])) 
    
#print(representative_frames)

# Make rep_frames directory
rf_dir='rep_conformers'
if not os.path.exists(rf_dir):
    os.mkdir(rf_dir)

np.savetxt(str(rf_dir)+'/'+str(analog)+'.dat',representative_frames, fmt='%d',delimiter=' ')

# Markov State Model

In [ ]:
#Choose which ring to analyze Cremer-Pople kinetics

ring_id=3
ring_theta=2*ring_id 
ring_phi=2*ring_id+1

#Choose which analogue
analog=3

#Feature array
Z=all_Z[analog-1]

## CLUSTER FEATURE DATA
#Choose number of clusters
n_clusters = 200   # number of k-means clusters
#Z feature object to Y conversions
Y=feature_obj(Z)
#Picks out the designated columns (feature subset) - used for MSM
MSM_dims_list=[ring_theta,ring_phi]
Yfeat=[Y[x][:,MSM_dims_list] for x in range(np.shape(Y)[0])]
# Cluster object
lambda_cl_obj = coor.cluster_kmeans(Yfeat, k=n_clusters, tolerance=1e-05, max_iter=100)
#Create discretized trajectories - where each snapshot is assigned to belong to a cluster e.g. microstate
lambda_dtrajs = lambda_cl_obj.dtrajs

## CALCULATE RELAXATION TIMESCALES
#Compute relaxation timescales
lambda_its = msm.timescales_msm(lambda_dtrajs, lags=1500, nits=5, errors='bayes')

## Build MARKOV MODEL
#Build MSM for chosen lag time
msm_lag = 1000
M = msm.bayesian_markov_model(lambda_dtrajs, msm_lag)
#Calculate stationary distribution of microstates and assign each snapshot with corresponding microstate density
M_stat=M.stationary_distribution
PI_all = np.hstack([M_stat[dtraj] for dtraj in M.discrete_trajectories_full])

# Eigenvector Projection
# project first 9 non-stationary eigenvectors
proj_ev_all = [np.hstack([M.eigenvectors_right()[:,i][dtraj] for dtraj in M.discrete_trajectories_full]) 
               for i in range(1, 10)]

#PCCA Analysis 
n_sets = 2
M.pcca(n_sets)
pccaX = M.metastable_distributions
pccaM = M.metastable_memberships  # get PCCA memberships
pcca_sets = M.metastable_sets
pcca_assign = M.metastable_assignments
# memberships and distributions over trajectory
X_all = [np.hstack([pccaX[i,:][dtraj] for dtraj in M.discrete_trajectories_full]) for i in range(n_sets)]
M_all = [np.hstack([pccaM[:,i][dtraj] for dtraj in M.discrete_trajectories_full]) for i in range(n_sets)]

#Hidden Markov Model Analysis
hmm = M.coarse_grain(2)
hmm_dist = hmm.metastable_distributions
hmm_membership = hmm.metastable_memberships  # get HMM memberships
# memberships over trajectory
hmm_dist_all = [np.hstack([hmm_dist[i,:][dtraj] for dtraj in hmm.discrete_trajectories_full]) for i in range(n_sets)]
hmm_mem_all = [np.hstack([hmm_membership[:,i][dtraj] for dtraj in hmm.discrete_trajectories_full]) for i in range(n_sets)]
hmm_sets = hmm.metastable_sets

#Compute k_for/k_rev kinetics
np.set_printoptions(precision=3,suppress=True)    
tau=10. 
rho=hmm.pi
delG=-0.596*np.log(rho)
delG=delG-np.max(delG)
P=hmm.transition_matrix
kon=1000.*P/tau
nd_kon=nondiag_rates(kon)

### Save kinetics/thermodynamics results to file

In [ ]:
#Concatenate thermodynamic/kinetic data in columns: From Macrostate, To Macrostate, k_for (us^-1), k_rev (us^-1), rho0, rho1, delG0, delG1 (max zeroed)  
out=np.hstack((nd_kon[0],rho,delG)).reshape(1,-1)
print(out)

# Make msm directory
msm_dir='msm'
if not os.path.exists(msm_dir):
    os.mkdir(msm_dir)

np.savetxt(str(msm_dir)+'/kon_'+str(analog)+'.dat',out, fmt='%6.2f',delimiter=' ')

### Plot graphs from MSM analysis

In [ ]:
# Make msm figure directory
msm_fig_dir=str(fig_dir)+'/msm'
if not os.path.exists(msm_fig_dir):
    os.mkdir(msm_fig_dir)

#### Plot Relaxation Timescales

In [ ]:
#Plot Timescale curves
plot_its(mplt,matplotlib,lambda_its,50,1000)

#save figure
save_current_fig(plt, str(msm_fig_dir)+'/its_' + str(analog) + '.png',8,6)

#### Plot PMF/Free Energy Landscape with G values and Kinetic Network

In [ ]:

MSM_dims=np.array(MSM_dims_list)
dim=[ring_phi,ring_theta]
#Weighted Free Energy PMF landscape with Contours
cl_obj=lambda_cl_obj
mstate_color=['yellow','red','magenta','blue','black']
matplotlib.rcParams.update({'font.size': 12})
plt=plot_weighted_free_energy_landscape_range(Z,plt,dim[0],dim[1],Zlabels,cmap='Spectral',wg=PI_all,lev_max=0,shallow=True,cbar=True,fsize=(11,9),x_lim=[0,210],y_lim=[60,190],cbar_label="PMF kcal/mol")

#Plot All Cluster centers
#plt.plot(cl_obj.clustercenters[:,np.where(MSM_dims==dim[0])[0][0]],cl_obj.clustercenters[:,np.where(MSM_dims==dim[1])[0][0]], linewidth=0, marker='o', markersize=8, color='k')

#Once PCCA/HMM sets has been calculated - Calculate clusters belonging to each PCCA/HMM metastable state
#plot_metastable_sets(plt,cl_obj,hmm_sets,MSM_dims,dim,mstate_color,msize=6)
#plot_metastable_sets(plt,cl_obj,pcca_sets,MSM_dims,dim,mstate_color,msize=6)

#State 0
plt.text(20, 150, '$^{1}C_{4}$', fontsize=40, color='k')
plt.text(20, 140, 'G = {:3.1f}'.format(delG[0]), fontsize=30, color='k')

#State 1 - Ground state
plt.text(20, 100, '$^{2}S_{O}$', fontsize=40, color='k')
plt.text(20, 90, 'G = {:3.1f}'.format(delG[1]), fontsize=30, color='k')

w=2
width_ratio=np.sqrt(nd_kon[0,2]/nd_kon[0,3])
plt.arrow(120,120,0,30,width=w,color='k',shape='right',length_includes_head=True,overhang=0.5)
plt.arrow(122,150,0,-30,width=w*width_ratio,color='k',shape='right',length_includes_head=True,overhang=0.5)


#State 0 -> 1
plt.text(135, 130, "{:3.1f}".format(nd_kon[0,2]), fontsize=30, color='k')
#State 1 -> 0
plt.text(95, 130, "{:3.1f}".format(nd_kon[0,3]), fontsize=30, color='k')

#save figure
save_current_fig(plt, str(msm_fig_dir)+'/pmf_' + str(analog) + '.png',11,9)

#### Plot Eigenvector Projections

In [ ]:
#Plot the fist 2 non-stationary eigenvectors binned into a 2D theta/phi CP space
fig, axes = subplots(1, n_sets, figsize=(16,6))
for i, ax in enumerate(axes):
    #plot_sampled_function(np.vstack(Y)[:,dim[0]], np.vstack(Y)[:,dim[1]], proj_ev_all[i], ax=ax, cbar=False, cmap=cm.Blues)
    plot_sampled_function(Z[:,dim[0]], Z[:,dim[1]], proj_ev_all[i], ax=ax, cbar=False, cmap=cm.Blues)
    #plot_labels(ax)

    
#save figure
save_current_fig(plt, str(msm_fig_dir)+'/eigenvectors_' + str(analog) + '.png',16,6)

#### Plot PCCA distributions

In [ ]:

fig, axes = subplots(1, n_sets, figsize=(16, 6))
matplotlib.rcParams.update({'font.size': 12})
axes = axes.flatten()
#import warnings
np.seterr(invalid='warn') 
for k in range(n_sets):
        plot_sampled_density(Z[:,dim[0]], Z[:,dim[1]], X_all[k], ax=axes[k], cmap="Spectral", cbar=False)     
        ax=axes[k]
        mstate_color=['yellow','red','magenta']
        ax.plot(cl_obj.clustercenters[pcca_sets[k],
np.where(MSM_dims==dim[0])[0][0]],cl_obj.clustercenters[pcca_sets[k],
np.where(MSM_dims==dim[1])[0][0]], linewidth=0, marker='o', markersize=2, color='black')

#save figure
save_current_fig(plt, str(msm_fig_dir)+'/pcca_' + str(analog) + '.png',16,6)


#### Plot HMM distributions

In [ ]:
fig, axes = subplots(1, 2, figsize=(16, 6))
matplotlib.rcParams.update({'font.size': 12})
axes = axes.flatten()
import warnings
np.seterr(invalid='warn') 
for k in range(n_sets):
        plot_sampled_density(Z[:,dim[0]], Z[:,dim[1]], hmm_dist_all[k], ax=axes[k], cmap="Spectral", cbar=False)     
        ax=axes[k]
        mstate_color=['yellow','red','magenta']
        ax.plot(cl_obj.clustercenters[hmm_sets[k],
np.where(MSM_dims==dim[0])[0][0]],cl_obj.clustercenters[hmm_sets[k],
np.where(MSM_dims==dim[1])[0][0]], linewidth=0, marker='o', markersize=2, color='black')

#save figure
save_current_fig(plt, str(msm_fig_dir)+'/hmm_' + str(analog) + '.png',16,6)



# End of Analysis

Correspondence: S. Kashif Sadiq (kashif.sadiq@embl.de) Affiliation: 1. Heidelberg Institute for Theoretical Studies, HITS gGmbH 2. European Moelcular Biology Laboratory